In [1]:
import torch

In [2]:
from torch.autograd import Variable

In [25]:
import  torch.nn as nn
import torch.nn.functional as F

In [6]:
x = Variable(torch.ones(2,2),requires_grad=True)
print(x)

tensor([[ 1.,  1.],
        [ 1.,  1.]])


In [7]:
print(x.data)
print(x.grad)
print(x.grad_fn)

tensor([[ 1.,  1.],
        [ 1.,  1.]])
None
None


In [8]:
y = x+2
print(y)

tensor([[ 3.,  3.],
        [ 3.,  3.]])


In [9]:
print(y.grad_fn)

In [10]:
z = y * y *3
out = z.mean()
print(z,out)


tensor([[ 27.,  27.],
        [ 27.,  27.]]) tensor(27.)


In [17]:
# out.backward()
print(x.grad)
# dout / dx

tensor([[ 4.5000,  4.5000],
        [ 4.5000,  4.5000]])


In [22]:
x = Variable(torch.ones((2,2)),requires_grad = True)
y = x + 2



In [23]:
y.backward(torch.ones(2,2),retain_graph = True)
print(x.grad)

tensor([[ 1.,  1.],
        [ 1.,  1.]])


In [24]:
gradient = torch.randn(2, 2)

y.backward(gradient=gradient)
print(x.grad)
torch.cat()

tensor([[ 2.3593, -0.1669],
        [ 1.6160,  1.7519]])


In [31]:
class RNN(nn.Module):
    def __init__(self,data_size, hidden_size, output_size):
        super(RNN,self).__init__()
        self.hidden_size = hidden_size
        input_size = data_size + hidden_size
        
        self.i2h = nn.Linear(input_size,hidden_size)
        self.h2o = nn.Linear(hidden_size,output_size)
        
    def forward(self,data,last_hidden):
        inputs = torch.cat((data,last_hidden),1)
        hidden = self.i2h(inputs)
        output = self.h2o(hidden)
        
        return hidden,output

        
    

In [32]:
rnn = RNN(50,20,10)

In [33]:
loss_fn = nn.MSELoss()

batch_size = 10
TIMESTEPS = 5
batch = Variable(torch.randn(batch_size, 50))
hidden = Variable(torch.zeros(batch_size, 20))
target = Variable(torch.zeros(batch_size, 10))

In [34]:
loss = 0 
for t in range(TIMESTEPS):
    hidden, output = rnn(batch,hidden)
    loss += loss_fn(output,target)
loss.backward()    

In [39]:
print(rnn.i2h.weight.grad.data)

tensor([[ 6.4866e-02, -7.4119e-02, -6.3060e-02,  ..., -3.4757e-02,
          1.0739e-02,  1.1959e-02],
        [ 5.9726e-02, -1.8990e-01,  1.5774e-02,  ..., -7.0121e-02,
          4.1341e-02, -2.1392e-03],
        [-8.9845e-02,  5.7795e-02, -8.4143e-02,  ...,  2.6410e-02,
         -1.3987e-02, -4.2004e-03],
        ...,
        [-2.5783e-02,  1.2292e-01,  2.9126e-02,  ...,  9.8170e-02,
         -7.4513e-03, -3.2644e-02],
        [ 1.2227e-02, -5.5118e-02, -8.5919e-02,  ..., -3.1665e-02,
          2.1021e-02,  9.3106e-03],
        [-4.6562e-02,  9.9821e-02, -4.4699e-03,  ...,  3.6211e-02,
         -4.0622e-02,  4.6574e-03]])


In [54]:
class MNISTConvNet(nn.Module):
    def __init__(self):
        super(MNISTConvNet,self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(10, 20, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self,inputs):
        x = self.pool1(F.relu(self.conv1(inputs)))
        x = self.pool2(F.relu(self.conv2(x)))
        
        x = x.view(x.size(0),-1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x

    
    

                
            

In [56]:
def printnorm(self,input,output):
        # input是将输入打包成的 tuple 的input
        # 输出是一个 Variable. output.data 是我们感兴趣的 Tensor
        print('Inside ' + self.__class__.__name__ + ' forward')
        print('')
        print('input: ', type(input))
        print('input[0]: ', type(input[0]))
        print('output: ', type(output))
        print('')
        print('input size:', input[0].size())
        print('output size:', output.data.size())
        print('output norm:', output.data.norm())    

        
def printgradnorm(self,grad_input,grad_output):
        print('Inside ' + self.__class__.__name__ + ' backward')
        print('Inside class:' + self.__class__.__name__)
        print('')
        print('grad_input: ', type(grad_input))
        print('grad_input[0]: ', type(grad_input[0]))
        print('grad_output: ', type(grad_output))
        print('grad_output[0]: ', type(grad_output[0]))
        print('')
        print('grad_input size:', grad_input[0].size())
        print('grad_output size:', grad_output[0].size())
        print('grad_input norm:', grad_input[0].data.norm())

In [57]:
net = MNISTConvNet()
net.conv2.register_forward_hook(printnorm)
print(net)

MNISTConvNet(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)


In [61]:
net.conv2.register_backward_hook(printgradnorm)
input = Variable(torch.randn(1, 1, 28, 28))
out = net(input)
print(out.size())

Inside Conv2d forward

input:  <class 'tuple'>
input[0]:  <class 'torch.Tensor'>
output:  <class 'torch.Tensor'>

input size: torch.Size([1, 10, 12, 12])
output size: torch.Size([1, 20, 8, 8])
output norm: tensor(14.8274)
torch.Size([1, 10])


In [62]:
target = Variable(torch.LongTensor([3]))
loss_fn = nn.CrossEntropyLoss()  # LogSoftmax + ClassNLL Loss
err = loss_fn(out, target)
err.backward()

Inside Conv2d backward
Inside class:Conv2d

grad_input:  <class 'tuple'>
grad_input[0]:  <class 'torch.Tensor'>
grad_output:  <class 'tuple'>
grad_output[0]:  <class 'torch.Tensor'>

grad_input size: torch.Size([1, 10, 12, 12])
grad_output size: torch.Size([1, 20, 8, 8])
grad_input norm: tensor(0.1266)
Inside Conv2d backward
Inside class:Conv2d

grad_input:  <class 'tuple'>
grad_input[0]:  <class 'torch.Tensor'>
grad_output:  <class 'tuple'>
grad_output[0]:  <class 'torch.Tensor'>

grad_input size: torch.Size([1, 10, 12, 12])
grad_output size: torch.Size([1, 20, 8, 8])
grad_input norm: tensor(0.1266)


In [49]:
print(err)

tensor(2.3118)


In [65]:
print(net.conv1.weight.grad.data)

tensor([[[[ 0.0478, -0.0476, -0.0388, -0.0287,  0.0257],
          [ 0.0099,  0.0529,  0.0834,  0.0067, -0.0230],
          [-0.0046, -0.0345, -0.0473,  0.0099, -0.0944],
          [-0.0271,  0.0146,  0.0570,  0.0069,  0.0040],
          [-0.0302,  0.0090, -0.0464, -0.0834, -0.0756]]],


        [[[ 0.0821, -0.0441, -0.1088,  0.0549, -0.0238],
          [ 0.0425, -0.0141, -0.0131,  0.0661, -0.1001],
          [-0.0532,  0.0104,  0.0697, -0.0445,  0.0013],
          [ 0.0152,  0.0056,  0.0217,  0.0595,  0.0443],
          [ 0.0113,  0.0292,  0.0647,  0.0437, -0.0347]]],


        [[[-0.0068, -0.0762,  0.0715,  0.0218,  0.0290],
          [-0.0352,  0.0114,  0.1271,  0.0534,  0.0514],
          [-0.0367,  0.0262, -0.0040,  0.0034, -0.0705],
          [ 0.0463,  0.0079,  0.0563, -0.0274,  0.0233],
          [ 0.0416,  0.0064, -0.0361,  0.0291, -0.0573]]],


        [[[-0.0098,  0.0478,  0.0428, -0.0507,  0.0298],
          [ 0.0033, -0.0395,  0.0012,  0.0803, -0.0116],
          [ 0.0042,

In [51]:
print(net.conv1.weight.data.norm()) 

tensor(1.8076)


In [63]:
print(net.conv1.weight )

Parameter containing:
tensor([[[[ 0.1955,  0.0979,  0.1875, -0.0746, -0.1249],
          [ 0.1798, -0.0377,  0.1087, -0.0187,  0.1748],
          [-0.0079,  0.1175, -0.1350,  0.0420, -0.1649],
          [-0.1567, -0.1062,  0.0849,  0.1565, -0.1799],
          [ 0.0658,  0.0009, -0.0490,  0.1356,  0.0889]]],


        [[[-0.0659,  0.1521,  0.0675,  0.0416,  0.0227],
          [-0.0665, -0.0617,  0.1824,  0.0383,  0.0557],
          [-0.1604, -0.0686,  0.0936,  0.0443,  0.1796],
          [-0.1764,  0.1220,  0.1382, -0.0644,  0.0217],
          [ 0.0408, -0.1252, -0.1183,  0.1791,  0.0248]]],


        [[[-0.1609, -0.0675, -0.1635,  0.1330,  0.0171],
          [ 0.1561, -0.1228, -0.1146, -0.0904, -0.1023],
          [ 0.1564, -0.1949, -0.1905,  0.0228,  0.0378],
          [ 0.1854, -0.1842, -0.1744,  0.1561, -0.0660],
          [-0.1391,  0.0213,  0.0758,  0.1615,  0.0583]]],


        [[[ 0.1589,  0.0013,  0.1320,  0.0520, -0.1452],
          [-0.0677, -0.1662, -0.1133, -0.1928,  0.0704